In [1]:
import NBInclude: @nbinclude
@nbinclude("RecommendationsBase.ipynb");

[ Info: 20220708 00:11:15 Insample RMSE: 1.2724406
[ Info: 20220708 00:11:15 Insample R2: 0.45511317


## Curated TV Series

In [2]:
first(
    rec_df |>
    tv_display |>
    implicit_display |>
    explicit_display |>
    unrelated_display |>
    unseen_display,
    50,
)

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,seen,related
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,5941,8550,Cross Game,"['Drama', 'Romance', 'School', 'Shounen', 'Sports', 'Team Sports']",tv,9.0,7.79907,0.00403852,0.876048,0.0188587,7.12028,9.87377e-5,9.0,0,0
2,27989,4600,Hibike! Euphonium,"['Drama', 'Music', 'Performing Arts', 'School']",tv,15.0,7.64189,0.00263704,0.997587,0.0175495,6.75024,0.000460287,15.0,0,0
3,1065,2038,Touch,"['Drama', 'Love Polygon', 'Romance', 'School', 'Shounen', 'Slice of Life', 'Sports', 'Team Sports']",tv,17.0,7.58733,0.000856546,0.720291,0.0175318,6.94463,1.9842e-5,17.0,0,0
4,122,5773,Full Moon wo Sagashite,"['Comedy', 'Drama', 'Music', 'Romance', 'Shoujo', 'Supernatural']",tv,24.0,7.28565,0.00166384,1.11776,0.0224149,6.30717,0.000147446,24.0,0,0
5,135,11918,Hikaru no Go,"['Comedy', 'Shounen', 'Strategy Game', 'Supernatural']",tv,27.0,7.48417,0.00112157,1.05287,0.01563,6.67032,0.000153084,27.0,0,0
6,42941,9316,Uma Musume: Pretty Derby (TV) Season 2,"['Anthropomorphic', 'Sports']",tv,30.0,7.66418,0.000188502,0.932386,0.0135477,6.65427,3.67776e-5,30.0,0,0
7,38993,11039,Karakai Jouzu no Takagi-san 2,"['Comedy', 'Iyashikei', 'Romantic Subtext', 'School', 'Shounen']",tv,37.0,7.34045,0.000956109,0.829321,0.0144625,6.69618,0.000452976,37.0,0,0
8,49721,14980,Karakai Jouzu no Takagi-san 3,"['Comedy', 'Iyashikei', 'Romantic Subtext', 'School', 'Shounen']",tv,43.0,7.41957,0.000683623,0.941736,0.015482,7.07822,0.000216592,43.0,0,0
9,25835,1337,Shirobako,"['Adult Cast', 'Comedy', 'Drama', 'Otaku Culture', 'Workplace']",tv,46.0,7.13735,0.00340941,0.610344,0.0192106,7.01304,0.000340545,46.0,0,0


## Curated Specials

In [3]:
first(
    rec_df |>
    inv_tv_display |>
    implicit_display |>
    explicit_display |>
    unrelated_display |>
    unseen_display,
    10,
)

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,seen,related
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,2454,15616,Kimagure Orange☆Road: Ano Hi ni Kaeritai,"['Comedy', 'Drama', 'Romance', 'Shounen', 'Slice of Life']",movie,21.0,7.59398,0.000175207,1.02539,0.0190324,6.59801,7.40786e-6,21.0,0,0
2,38422,13888,High Score Girl: Extra Stage,"['Comedy', 'Love Polygon', 'Romantic Subtext', 'School', 'Seinen', 'Video Game']",ova,28.0,7.51018,0.000960544,1.16391,0.0157395,6.74174,9.15347e-5,28.0,0,0
3,40664,19524,Shoujo☆Kageki Revue Starlight Movie,"['Music', 'Performing Arts', 'School']",movie,29.0,7.76013,0.000188372,0.949996,0.0164447,7.68836,7.09105e-6,29.0,0,0
4,35677,18606,Liz to Aoi Tori,"['Drama', 'Music', 'Performing Arts', 'School']",movie,51.0,7.10951,0.00313026,0.77692,0.022326,7.10863,0.000104203,51.0,0,0
5,35678,12875,Hibike! Euphonium Movie 3: Chikai no Finale,"['Drama', 'Music', 'Performing Arts', 'School']",movie,52.0,7.22077,0.000820769,0.747372,0.0153225,6.57013,4.78398e-5,52.0,0,0
6,949,9788,Top wo Nerae! Gunbuster,"['Action', 'Comedy', 'Drama', 'Mecha', 'Military', 'Sci-Fi', 'Shounen', 'Space']",ova,54.0,7.06907,0.00448864,0.698275,0.0224796,7.01077,0.000117517,54.0,0,0
7,44042,9155,Holo no Graffiti,['Comedy'],ona,58.0,7.42197,6.37469e-5,0.945167,0.0155154,7.14345,6.306e-5,58.0,0,0
8,15989,7399,Saki Achiga-hen: Episode of Side-A Specials,"['School', 'Slice of Life', 'Strategy Game']",special,59.0,7.34375,0.000247256,0.950948,0.0127529,6.26454,1.09786e-5,59.0,0,0
9,34085,9971,Sakura Nagashi,"['Action', 'Drama', 'Mecha', 'Music', 'Sci-Fi']",music,85.0,7.29159,6.40274e-5,1.2635,0.0150664,6.23364,5.13897e-6,85.0,0,0


## Continue Watching TV Series

In [4]:
first(rec_df |> tv_display |> related_display |> unseen_display, 10)

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,seen,related
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,42587,13295,Yuuki Yuuna wa Yuusha de Aru: Dai Mankai no Shou,"['Drama', 'Fantasy', 'Mahou Shoujo', 'Slice of Life']",tv,7.0,8.06117,0.000601653,0.974526,0.0201745,5.94387,1.02947e-5,7.0,0,1
2,43608,7264,Kaguya-sama wa Kokurasetai: Ultra Romantic,"['Comedy', 'Psychological', 'Romantic Subtext', 'School', 'Seinen', 'Suspense']",tv,22.0,7.7963,0.000948903,0.941139,0.0151677,7.47467,0.00101131,22.0,0,1
3,37999,3442,Kaguya-sama wa Kokurasetai: Tensai-tachi no Renai Zunousen,"['Comedy', 'Psychological', 'Romantic Subtext', 'School', 'Seinen']",tv,34.0,7.10852,0.00429605,0.672193,0.0288182,7.05217,0.00342105,34.0,0,1
4,33,18323,Kenpuu Denki Berserk,"['Action', 'Adventure', 'Drama', 'Fantasy', 'Gore', 'Horror', 'Military', 'Mythology', 'Seinen', 'Supernatural']",tv,38.0,7.17114,0.00104412,0.754057,0.0251038,7.30844,0.000996253,38.0,0,1
5,23623,14046,Non Non Biyori Repeat,"['CGDCT', 'Iyashikei', 'School', 'Seinen', 'Slice of Life']",tv,53.0,7.12724,0.00236792,0.568073,0.0160314,6.86341,0.000215291,53.0,0,1
6,5114,17079,Fullmetal Alchemist: Brotherhood,"['Action', 'Adventure', 'Drama', 'Fantasy', 'Military', 'Shounen']",tv,55.0,7.05038,0.00233275,0.99369,0.0170687,7.66311,0.00833421,55.0,0,1
7,39808,17939,Non Non Biyori Nonstop,"['CGDCT', 'Iyashikei', 'Seinen', 'Slice of Life']",tv,60.0,7.13531,0.00161622,0.632685,0.0170845,7.08152,9.58043e-5,60.0,0,1
8,11239,14609,Hidamari Sketch x Honeycomb,"['CGDCT', 'Comedy', 'Iyashikei', 'School', 'Seinen', 'Slice of Life', 'Visual Arts']",tv,69.0,7.09316,0.000887204,0.612616,0.0159337,6.90358,1.45948e-5,69.0,0,1
9,19111,15311,Love Live! School Idol Project 2nd Season,"['Idols (Female)', 'Music', 'School', 'Slice of Life']",tv,71.0,7.03824,0.00063574,0.651091,0.0182952,6.3506,0.000406483,71.0,0,1


## Continue Watching Specials

In [5]:
first(rec_df |> inv_tv_display |> related_display |> unseen_display, 10)

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,seen,related
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,11979,14471,Mahou Shoujo Madoka★Magica Movie 2: Eien no Monogatari,"['Drama', 'Mahou Shoujo', 'Psychological', 'Suspense']",movie,1.0,9.04965,0.00172292,0.955487,0.0197045,7.0972,0.000228368,1.0,0,1
2,34376,1289,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 3 - Yakusoku,"['Drama', 'Fantasy', 'Mahou Shoujo', 'Slice of Life']",movie,2.0,9.03747,0.000198568,0.89334,0.012472,6.84503,6.8073e-6,2.0,0,1
3,11977,11665,Mahou Shoujo Madoka★Magica Movie 1: Hajimari no Monogatari,"['Drama', 'Mahou Shoujo', 'Psychological', 'Suspense']",movie,3.0,8.52262,0.00159649,0.931889,0.0220222,6.93514,0.000234344,3.0,0,1
4,34374,7098,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 1 - Tomodachi,"['Drama', 'Fantasy', 'Mahou Shoujo', 'Slice of Life']",movie,4.0,8.4127,0.000171636,1.15296,0.0151327,6.33698,6.85589e-6,4.0,0,1
5,34375,2707,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 2 - Tamashii,"['Drama', 'Fantasy', 'Mahou Shoujo', 'Slice of Life']",movie,5.0,8.42054,0.000253027,0.997434,0.015898,6.64496,6.78558e-6,5.0,0,1
6,31,17299,Neon Genesis Evangelion: Death & Rebirth,"['Drama', 'Mecha', 'Psychological', 'Sci-Fi']",movie,8.0,7.91132,0.000733261,0.945158,0.0175607,6.15442,0.000425672,8.0,0,1
7,4059,728,"Clannad: Mou Hitotsu no Sekai, Tomoyo-hen","['Drama', 'Romance', 'School']",special,11.0,7.65899,0.00244737,0.90061,0.0232225,6.50166,0.000512151,11.0,0,1
8,371,7437,Cardcaptor Sakura Movie 1,"['Comedy', 'Drama', 'Fantasy', 'Mahou Shoujo', 'Shoujo']",movie,12.0,7.69652,0.00237173,0.806992,0.0166514,6.1302,0.000153506,12.0,0,1
9,6862,2353,K-On!: Live House!,"['CGDCT', 'Comedy', 'Music', 'School']",special,13.0,7.63737,0.000721989,0.944511,0.022394,6.4686,0.000246322,13.0,0,1


## Popular items

In [6]:
first(
    rec_df |> tv_display |> inv_implicit_display |> unrelated_display |> unseen_display,
    10,
)

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,seen,related
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,38524,12753,Shingeki no Kyojin Season 3 Part 2,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,6.0,8.09787,0.00101759,0.684831,0.0138733,7.72105,0.00602488,6.0,0,0
2,40028,5414,Shingeki no Kyojin: The Final Season,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,10.0,7.81374,0.000751328,0.811394,0.0125314,7.44496,0.00547408,10.0,0,0
3,25777,8685,Shingeki no Kyojin Season 2,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,14.0,7.60877,0.00119347,0.911592,0.0149581,7.13599,0.00695737,14.0,0,0
4,16498,7644,Shingeki no Kyojin,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,18.0,7.22825,0.00231433,1.03395,0.017008,7.08161,0.0115216,18.0,0,0
5,11061,7814,Hunter x Hunter (2011),"['Action', 'Adventure', 'Fantasy', 'Shounen']",tv,19.0,7.52644,0.000765398,0.95483,0.0141401,7.61258,0.00679227,19.0,0,0
6,35760,19030,Shingeki no Kyojin Season 3,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,23.0,7.38798,0.0011309,0.819895,0.0160394,7.26902,0.00601985,23.0,0,0
7,48583,18963,Shingeki no Kyojin: The Final Season Part 2,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,31.0,7.57007,0.000373345,0.792355,0.0116661,7.49289,0.00303241,31.0,0,0
8,50265,4347,Spy x Family,"['Action', 'Childcare', 'Comedy', 'Shounen']",tv,42.0,7.28062,0.000944774,0.972967,0.0154375,7.51584,0.00329886,42.0,0,0
9,479,12288,Ueki no Housoku,"['Action', 'Adventure', 'Comedy', 'Drama', 'Shounen', 'Super Power', 'Supernatural']",tv,83.0,5.79514,9.48514e-5,1.146,0.0526402,6.1745,0.000123483,83.0,0,0


## Highly Rated Items

In [7]:
first(
    rec_df |> tv_display |> inv_explicit_display |> unrelated_display |> unseen_display,
    10,
)

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,seen,related
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,11061,7814,Hunter x Hunter (2011),"['Action', 'Adventure', 'Fantasy', 'Shounen']",tv,19.0,7.52644,0.000765398,0.95483,0.0141401,7.61258,0.00679227,19.0,0,0
2,50265,4347,Spy x Family,"['Action', 'Childcare', 'Comedy', 'Shounen']",tv,42.0,7.28062,0.000944774,0.972967,0.0154375,7.51584,0.00329886,42.0,0,0
3,46102,6211,Odd Taxi,"['Anthropomorphic', 'Mystery']",tv,50.0,7.07394,0.00254737,0.671028,0.0243766,7.62744,0.000606683,50.0,0,0
4,2164,4538,Dennou Coil,"['Adventure', 'Comedy', 'Drama', 'Mystery', 'Sci-Fi']",tv,81.0,6.8674,0.00641436,0.900115,0.0178745,6.92828,0.000101294,81.0,0,0
5,479,12288,Ueki no Housoku,"['Action', 'Adventure', 'Comedy', 'Drama', 'Shounen', 'Super Power', 'Supernatural']",tv,83.0,5.79514,9.48514e-5,1.146,0.0526402,6.1745,0.000123483,83.0,0,0
6,38474,11711,Yuru Camp△ Season 2,"['CGDCT', 'Iyashikei', 'Slice of Life']",tv,93.0,6.95893,0.00161795,0.823942,0.0187276,7.26937,0.000217368,93.0,0,0
7,40815,5820,Honzuki no Gekokujou: Shisho ni Naru Tame ni wa Shudan wo Erandeiraremasen 2nd Season,"['Fantasy', 'Isekai', 'Reincarnation', 'Slice of Life']",tv,122.0,6.76008,0.00104476,0.967247,0.021598,6.772,0.000222718,122.0,0,0
8,37510,7710,Mob Psycho 100 II,"['Action', 'Comedy', 'Super Power', 'Supernatural']",tv,127.0,6.85942,0.000981002,0.861961,0.0173322,7.54907,0.00285437,127.0,0,0
9,47778,18684,Kimetsu no Yaiba: Yuukaku-hen,"['Action', 'Fantasy', 'Historical', 'Shounen']",tv,135.0,6.94916,0.000488577,1.0016,0.0152367,7.46346,0.00323079,135.0,0,0
